In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torchtext.vocab import GloVe,FastText
import spacy
import io
import jsonlines
import json
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import random_split
import fasttext
import re
from spacy.language import Language
from spacy.tokens import Doc

cpu = torch.device('cpu')

if torch.has_mps:
    device = torch.device('mps')
else:
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')


EMBED_DIM = 300
HIDDEN_LAYER_DIM = 100
NUM_EPOCHS = 40

In [10]:
model = fasttext.load_model('./crawl-300d-2M-subword/crawl-300d-2M-subword.bin')

In [ ]:
model.get_word_vector('21st')

In [ ]:
f = FastText()

In [ ]:
f.__getitem__('21st')

In [ ]:
g = GloVe()


In [ ]:
g.__getitem__('21st')

In [11]:
@Language.component("expandcontractions")
def expand_contractions(text):
    
    
    flags = re.IGNORECASE | re.MULTILINE
    text = re.sub(r'`', "'", text, flags = flags)
    
    ## starts / ends with '
    text = re.sub(
        r"(\s|^)'(aight|cause)(\s|$)",
        '\g<1>\g<2>\g<3>',
        text, flags = flags
    )
    
    text = re.sub(
        r"(\s|^)'t(was|is)(\s|$)", r'\g<1>it \g<2>\g<3>',
        text,
        flags = flags
    )
    
    text = re.sub(
        r"(\s|^)ol'(\s|$)",
        '\g<1>old\g<2>',
        text, flags = flags
    )
    
    text = re.sub(r"\b(aight)\b", 'alright', text, flags = flags)
    text = re.sub(r'\bcause\b', 'because', text, flags = flags)
    text = re.sub(r'\b(finna|gonna)\b', 'going to', text, flags = flags)
    text = re.sub(r'\bgimme\b', 'give me', text, flags = flags)
    text = re.sub(r"\bgive'n\b", 'given', text, flags = flags)
    text = re.sub(r"\bhowdy\b", 'how do you do', text, flags = flags)
    text = re.sub(r"\bgotta\b", 'got to', text, flags = flags)
    text = re.sub(r"\binnit\b", 'is it not', text, flags = flags)
    text = re.sub(r"\b(can)(not)\b", r'\g<1> \g<2>', text, flags = flags)
    text = re.sub(r"\bwanna\b", 'want to', text, flags = flags)
    text = re.sub(r"\bmethinks\b", 'me thinks', text, flags = flags)
    text = re.sub(r"\bo'er\b", r'over', text, flags = flags)
    text = re.sub(r"\bne'er\b", r'never', text, flags = flags)
    text = re.sub(r"\bo'?clock\b", 'of the clock', text, flags = flags)
    text = re.sub(r"\bma'am\b", 'madam', text, flags = flags)
    text = re.sub(r"\bgiv'n\b", 'given', text, flags = flags)
    text = re.sub(r"\be'er\b", 'ever', text, flags = flags)
    text = re.sub(r"\bd'ye\b", 'do you', text, flags = flags)
    text = re.sub(r"\be'er\b", 'ever', text, flags = flags)
    text = re.sub(r"\bd'ye\b", 'do you', text, flags = flags)
    text = re.sub(r"\bg'?day\b", 'good day', text, flags = flags)
    text = re.sub(r"\b(ain|amn)'?t\b", 'am not', text, flags = flags)
    text = re.sub(r"\b(are|can)'?t\b", r'\g<1> not', text, flags = flags)
    text = re.sub(r"\b(let)'?s\b", r'\g<1> us', text, flags = flags)
    text = re.sub(r"\by'all'dn't've'd\b", 'you all would not have had', text, flags = flags)
    text = re.sub(r"\by'all're\b", 'you all are', text, flags = flags)
    text = re.sub(r"\by'all'd've\b", 'you all would have', text, flags = flags)
    text = re.sub(r"(\s)y'all(\s)", r'\g<1>you all\g<2>', text, flags = flags)  
    text = re.sub(r"\b(won)'?t\b", 'will not', text, flags = flags)
    text = re.sub(r"\bhe'd\b", 'he had', text, flags = flags)
    text = re.sub(r"\b(I|we|who)'?d'?ve\b", r'\g<1> would have', text, flags = flags)
    text = re.sub(r"\b(could|would|must|should|would)n'?t'?ve\b", r'\g<1> not have', text, flags = flags)
    text = re.sub(r"\b(he)'?dn'?t'?ve'?d\b", r'\g<1> would not have had', text, flags = flags)
    text = re.sub(r"\b(daren|daresn|dasn)'?t", 'dare not', text, flags = flags)
    text = re.sub(r"\b(he|how|i|it|she|that|there|these|they|we|what|where|which|who|you)'?ll\b", r'\g<1> will', text, flags = flags)
    text = re.sub(r"\b(everybody|everyone|he|how|it|she|somebody|someone|something|that|there|this|what|when|where|which|who|why)'?s\b", r'\g<1> is', text, flags = flags)
    text = re.sub(r"\b(I)'?m'a\b", r'\g<1> am about to', text, flags = flags)
    text = re.sub(r"\b(I)'?m'o\b", r'\g<1> am going to', text, flags = flags)
    text = re.sub(r"\b(I)'?m\b", r'\g<1> am', text, flags = flags)
    text = re.sub(r"\bshan't\b", 'shall not', text, flags = flags)
    text = re.sub(r"\b(are|could|did|does|do|go|had|has|have|is|may|might|must|need|ought|shall|should|was|were|would)n'?t\b", r'\g<1> not', text, flags = flags)
    text = re.sub(r"\b(could|had|he|i|may|might|must|should|these|they|those|to|we|what|where|which|who|would|you)'?ve\b", r'\g<1> have', text, flags = flags)
    text = re.sub(r"\b(how|so|that|there|these|they|those|we|what|where|which|who|why|you)'?re\b", r'\g<1> are', text, flags = flags)
    text = re.sub(r"\b(I|it|she|that|there|they|we|which|you)'?d\b", r'\g<1> had', text, flags = flags)
    text = re.sub(r"\b(how|what|where|who|why)'?d\b", r'\g<1> did', text, flags = flags)
    
    return text


In [12]:
nlp = spacy.load("en_core_web_sm")
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x436614220>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x111feb760>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x4364c1f20>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x15770d980>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x15775f700>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x436600040>)]

In [13]:


def preprocess_text(text):    
    words = nlp(text)
    #sentence = [token.text_with_ws for token in words if not token.is_punct]
    sentence  = "".join([token.text_with_ws for token in words if not token.is_punct ]).strip()
    sentence = sentence.lower()
    return sentence

def process_training_data():
    negative_reviews = io.open('./Train.neg',encoding='latin-1').readlines()
    positive_reviews = io.open('./Train.pos',encoding='latin-1').readlines()
    with jsonlines.open('train.jsonl',mode='w') as writer:

        for review in positive_reviews:
            processed_text = preprocess_text(review)
            d = {'text': processed_text , 'sentiment': 1}
            writer.write(d)
        for review in negative_reviews:
            processed_text = preprocess_text(review)
            d = {'text': processed_text , 'sentiment': 0}
            writer.write(d)
process_training_data()


In [14]:
from torch.utils.data import DataLoader,Dataset

class ReviewDataSet(Dataset):

    def __init__(self,file):
        super().__init__()
        
        self.file = file
        self.data = []
        with open(self.file) as f:
            for line in f:
                sample = json.loads(line)
                self.data.append([sample['text'],sample['sentiment']])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]

    

dataset = ReviewDataSet('train.jsonl') 

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
print(train_size)
print(test_size)

train_dataset,validation_dataset = random_split(dataset,[train_size,test_size])

train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle=True)
val_dataloader = DataLoader(validation_dataset,batch_size=32,shuffle=True)
glove = GloVe()


8000
2000


In [15]:
def getWordEmbeddings(batch_data,glove):
    
    
    if(len(batch_data)==2):
        reviews = batch_data[0]
        sentiment = batch_data[1]
    else:
        reviews = batch_data[0]
        sentiment = None
    
    reviews_tensor = []
    lengths = []
    for review in reviews:
        words = review.split()
        words_tensor = []
        lengths.append(len(words))
        for word in words:
            #words_tensor.append(glove.__getitem__(word))
            words_tensor.append(torch.tensor(glove.get_word_vector(word)))
        reviews_tensor.append(torch.stack(words_tensor,dim=0))
    
    mask = torch.zeros((len(lengths),max(lengths)))
    for i in range(len(lengths)):
        mask[i,:lengths[i]] = 1.0
    
    return (pad_sequence(reviews_tensor,batch_first=True),mask,sentiment)




In [16]:
'''from gensim import models

w = models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin',binary=True)
w.get_vector('rat')
'''

"from gensim import models\n\nw = models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin',binary=True)\nw.get_vector('rat')\n"

In [17]:
class DAN(nn.Module):

    def __init__(self,embed_dim=EMBED_DIM,hidden_dim = HIDDEN_LAYER_DIM):
        super().__init__()
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.fc1 = nn.Linear(self.embed_dim,self.hidden_dim)
        self.fc = nn.Linear(self.hidden_dim,self.hidden_dim)
        self.fc2 = nn.Linear(self.hidden_dim,1)
        self.sigmoid  = nn.Sigmoid()

    def forward(self,inp,inp_mask):
        
        inp_lengths = torch.sum(inp_mask,-1,keepdim=True)
        total = torch.sum(inp*(inp_mask.unsqueeze(2)),axis=1)
        vector_average = total / inp_lengths
        ans = F.relu(self.fc1(vector_average))
        ans = F.relu(self.fc(ans))
        ans = self.sigmoid(self.fc2(ans))
        return ans
    


In [18]:
'''inp = torch.randn((2,3,5))
mask = torch.tensor([[1.0,0.0,0.0],[1.0,0.0,1.0]])

print(inp)'''

'inp = torch.randn((2,3,5))\nmask = torch.tensor([[1.0,0.0,0.0],[1.0,0.0,1.0]])\n\nprint(inp)'

In [19]:
# Training loop
dan = DAN(EMBED_DIM,HIDDEN_LAYER_DIM)

criterion = nn.BCELoss()
optimizer = optim.Adam(dan.parameters(),lr=0.01)

for e in range(NUM_EPOCHS):
    training_loss = 0.0
    size = 0
    dan.train()

    for i,data in enumerate(train_dataloader,0):
        
        optimizer.zero_grad()
        
        input_reviews , input_mask , output_labels = getWordEmbeddings(data,model)

        output = dan(input_reviews,input_mask).squeeze()
        
     

        loss = criterion(output,output_labels.float())
       
        
        
        training_loss += loss.item()
        loss.backward()
        optimizer.step()
        size = max(size,i+1)

    dan.eval()
    validation_loss = 0

    val_size = 0
    for i,data in enumerate(val_dataloader,0):
        input_reviews,input_mask,output_labels = getWordEmbeddings(data,model)
        output = dan(input_reviews,input_mask).squeeze()
        nearest_class = torch.round(output)
        correct = (nearest_class == output_labels.float()).float()
        validation_loss += correct.sum()
  
    print(str(training_loss/size )+ "   " + str(validation_loss/len(validation_dataset)))


0.5609703376293182   tensor(0.7395)
0.4992580144405365   tensor(0.7590)
0.47869559985399246   tensor(0.7540)
0.46793403470516204   tensor(0.7595)
0.46379221087694167   tensor(0.7725)
0.4575304556488991   tensor(0.6840)
0.45060817152261734   tensor(0.7760)
0.4531295042037964   tensor(0.7455)
0.4385815027952194   tensor(0.7705)
0.43537878841161726   tensor(0.7735)
0.431089332818985   tensor(0.7395)
0.42689629596471784   tensor(0.7650)
0.42459876781702044   tensor(0.7745)
0.41615037870407107   tensor(0.7715)
0.4116773464083672   tensor(0.7755)
0.40364035159349443   tensor(0.7755)
0.4008920320868492   tensor(0.7740)
0.39158280968666076   tensor(0.7570)
0.392457572221756   tensor(0.7555)
0.3937276052236557   tensor(0.7600)
0.3827943657040596   tensor(0.7610)
0.3743901337981224   tensor(0.7635)
0.3682677587270737   tensor(0.7595)
0.3656094669103622   tensor(0.7555)
0.3640474045276642   tensor(0.7675)
0.3562387942671776   tensor(0.7670)
0.3492117664217949   tensor(0.7660)
0.3492134701013565  

In [ ]:
def test(filename):
    reviews = open(filename,'r').readlines()
    for i in range(len(reviews)):
        r = reviews[i]
        reviews[i] = preprocess_text(r)
    
    reviews,reviews_mask,labels = getWordEmbeddings([reviews],model)
    dan.eval()

    output = dan(reviews,reviews_mask)
    print(output)


test('test.txt')
